## Load Data

In [1]:
rawData = readlines("oceanFloor.txt");

In [14]:
function convertInput(input::String)
    len = length(input)
    conv = Array{Int32}(undef,len)
    for i in 1:len
        if input[i] == '.'
            conv[i] = 0
        else
            conv[i] = 1
        end
    end
    return conv
end

convertInput (generic function with 1 method)

In [19]:
mask = convertInput(rawData[1]);
image = Array{Int32}(undef,length(rawData[3:end]),length(rawData[3]))
for (i,s) in enumerate(rawData[3:end])
    image[i,:] = convertInput(s);
end

In [32]:
rawDataTest = "..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..###..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#..#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#......#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#.....####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.......##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#

#..#.
#....
##..#
..#..
..###";

In [33]:
rawDataTest=String.(split(rawDataTest,"\n"))
maskTest = convertInput(rawDataTest[1]);
imageTest = Array{Int32}(undef,length(rawDataTest[3:end]),length(rawDataTest[3]))
for (i,s) in enumerate(rawDataTest[3:end])
    imageTest[i,:] = convertInput(s);
end

## Part 1

In [66]:
function enhanceImage(inImage::Array{Int32,2},mask::Array{Int32,1};npass=1)
    inSize = size(inImage)
    # account for extended image turning 1 every second pass if mask[0]==1
    if mask[1] == 1 && iseven(npass)
        origImage = ones(Int32,inSize[1]+4,inSize[2]+4)
    else
        origImage = zeros(Int32,inSize[1]+4,inSize[2]+4)
    end
    origImage[3:end-2,3:end-2] = inImage
    outImage = Array{Int32}(undef,inSize[1]+2,inSize[2]+2)
    
    subImage = Array{Int32}(undef,3,3)
    for i in 2:inSize[1]+3, j in 2:inSize[2]+3
        
        for k in -1:1,l in -1:1
            #store subImage transposed so information is already fortran contiguous for later use
            subImage[l+2,k+2] = origImage[i+k,j+l]
        end
        
        value = parse(Int,join(string.(subImage)),base=2) #turn image into single binary number and convert to base 10
        outImage[i-1,j-1] = mask[value+1] # account for 1-based indexing
    end
    return outImage
end

enhanceImage (generic function with 1 method)

In [46]:
imageTest

5×5 Array{Int32,2}:
 1  0  0  1  0
 1  0  0  0  0
 1  1  0  0  1
 0  0  1  0  0
 0  0  1  1  1

In [69]:
pass1t = enhanceImage(imageTest,maskTest)

7×7 Array{Int32,2}:
 0  1  1  0  1  1  0
 1  0  0  1  0  1  0
 1  1  0  1  0  0  1
 1  1  1  1  0  0  1
 0  1  0  0  1  1  0
 0  0  1  1  0  0  1
 0  0  0  1  0  1  0

In [70]:
pass2t = enhanceImage(pass1t,maskTest)

9×9 Array{Int32,2}:
 0  0  0  0  0  0  0  1  0
 0  1  0  0  1  0  1  0  0
 1  0  1  0  0  0  1  1  1
 1  0  0  0  1  1  0  1  0
 1  0  0  0  0  0  1  0  1
 0  1  0  1  1  1  1  1  0
 0  0  1  0  1  1  1  1  1
 0  0  0  1  1  0  1  1  0
 0  0  0  0  1  1  1  0  0

In [71]:
sum(pass2t)

35

In [72]:
passN = image
for i in 1:2
    passN = enhanceImage(passN,mask,npass=i)
end
sum(passN)

5347

## Part 2

In [73]:
passN = image
for i in 1:50
    passN = enhanceImage(passN,mask,npass=i)
end
sum(passN)

17172